In [2]:
import pandas as pd
import numpy as np
from lifelines import CoxPHFitter
from sklearn.metrics import log_loss
from sklearn.metrics import brier_score_loss

In [3]:
df_train = pd.read_csv('df_train.csv')
df_test = pd.read_csv('df_test.csv')

In [4]:
# CPH0

In [5]:
df_train_cph0 = df_train.copy()
df_test_cph0 = df_test.copy()

In [6]:
cph = CoxPHFitter()
cph.fit(df_train_cph0, duration_col='TIMETOEVENT', event_col='MORTSTAT')

cph.print_summary()

<lifelines.CoxPHFitter: fitted with 82720 total observations, 78101 right-censored observations>
             duration col = 'TIMETOEVENT'
                event col = 'MORTSTAT'
      baseline estimation = breslow
   number of observations = 82720
number of events observed = 4619
   partial log-likelihood = -45724.40
         time fit was run = 2025-03-08 22:04:53 UTC

---
              coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                     
REGION        0.02      1.02      0.01           -0.01            0.05                0.99                1.05
AGE           0.07      1.08      0.00            0.07            0.08                1.07                1.08
SEX          -0.39      0.68      0.03           -0.45           -0.33                0.63                0.72
MARST         0.01      1.01      0.00            0.01            0.01                1.01                1.01
FAMSIZE      -0.02      0.98      0.02           -0.06            0.01                0.95                1.01
RACEA        -0.04      0.96      0.03           -0.09            0.01                0.91                1.01
USBORN        0.04      1.04      0.01            0.03            0.05                1.03                1.06
EDUCREC1      0.01      1.01      0.01           -0.01            0.02                0.99                1.02
POVERTY      -0.10      0.90      0.01           -0.12           -0.08                0.88                0.92
FSRAWSCORE    0.00      1.00      0.01           -0.01            0.02                0.99                1.02
BMICALC      -0.02      0.98      0.00           -0.03           -0.01                0.97                0.99
HSTATYR       0.12      1.13      0.03            0.06            0.17                1.06                1.19
HINOTCOVE     0.09      1.10      0.08           -0.07            0.26                0.93                1.29
HIPRIVATEE   -0.05      0.95      0.04           -0.12            0.02                0.89                1.02
HIMCAIDE      0.10      1.10      0.05            0.00            0.19                1.00                1.21
HIMCAREE      0.18      1.20      0.05            0.08            0.28                1.08                1.33
ARTHGLUPEV   -0.02      0.98      0.03           -0.08            0.05                0.92                1.05
BLIND         0.64      1.89      0.11            0.43            0.85                1.53                2.34
CANCEREV      0.31      1.36      0.03            0.24            0.38                1.27                1.46
CHEARTDIEV    0.18      1.20      0.04            0.10            0.27                1.10                1.31
CPOXEV       -0.10      0.90      0.04           -0.17           -0.03                0.84                0.97
DIABETICEV    0.19      1.21      0.04            0.12            0.26                1.12                1.30
EMPHYSEMEV    0.32      1.38      0.06            0.21            0.43                1.24                1.54
HEARTATTEV    0.14      1.15      0.05            0.04            0.24                1.04                1.27
HEARTCONEV    0.25      1.29      0.04            0.18            0.33                1.20                1.39
HYPERTENEV    0.11      1.11      0.03            0.04            0.17                1.04                1.19
KIDNEYWKYR    0.43      1.54      0.05            0.33            0.54                1.39                1.71
LIVERCHRON    0.36      1.44      0.10            0.17            0.56                1.18                1.75
LIVERCONYR    0.28      1.32      0.10            0.09            0.47                1.09                1.59
STROKEV       0.21      1.24      0.05            0.12            0.30                1.13                1.36
ALC5UPYR      0.00      1.00      0.00            0.00            0.00  

In [7]:
print(f'APLL_is: {cph.score(df_train_cph0, scoring_method="log_likelihood"):.4f}')
print(f'APLL_os: {cph.score(df_test_cph0, scoring_method="log_likelihood"):.4f}')
print(f'C_is: {cph.score(df_train_cph0, scoring_method="concordance_index"):.4f}')
print(f'C_os: {cph.score(df_test_cph0, scoring_method="concordance_index"):.4f}')

APLL_is: -0.5528
APLL_os: -0.4629
C_is: 0.8817
C_os: 0.8849


In [8]:
# predict the survival function for each individual
survival_probs_16_train = cph.predict_survival_function(df_train_cph0)
survival_probs_16_test = cph.predict_survival_function(df_test_cph0)

# extract 3y survival probabilities
survival_prob_3y_train = survival_probs_16_train.iloc[3] 
survival_prob_3y_test = survival_probs_16_test.iloc[3] 

# get 3-year event status based on TIMETOEVENT and MORTSTAT
status_3y_train = np.where((df_train_cph0["MORTSTAT"] == 1) & (df_train_cph0["TIMETOEVENT"] <= 3), 1, 0)
status_3y_test = np.where((df_test_cph0["MORTSTAT"] == 1) & (df_test_cph0["TIMETOEVENT"] <= 3), 1, 0)

print(f'log-loss is: {log_loss(status_3y_train, 1-survival_prob_3y_train):.4f}')
print(f'log-loss os: {log_loss(status_3y_test, 1-survival_prob_3y_test):.4f}')
print(f'BS is: {brier_score_loss(status_3y_train, 1-survival_prob_3y_train):.4f}')
print(f'BS os: {brier_score_loss(status_3y_test, 1-survival_prob_3y_test):.4f}')

log-loss is: 0.1137
log-loss os: 0.1118
BS is: 0.0299
BS os: 0.0296


In [9]:
# save for next model
df_train_cph0.to_csv("df_train_cph0.csv", index=False)
df_test_cph0.to_csv("df_test_cph0.csv", index=False)